In [1]:
import pandas as pd

In [2]:
# Load necessary data files
admissions = 'C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/preprocess/admissions_4.csv'
omr = 'C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/omr.csv'

In [3]:
import pandas as pd

# Baca file CSV dalam chunk
chunk_size = 100000  # Jumlah baris per chunk
chunks = pd.read_csv(omr, chunksize=chunk_size)

# Proses setiap chunk
for chunk in chunks:
    # Lakukan pemrosesan pada setiap chunk
    print(chunk.head())  # Contoh: tampilkan 5 baris pertama dari setiap chunk

   subject_id   chartdate  seq_num      result_name result_value
0    10000032  2180-04-27        1   Blood Pressure       110/65
1    10000032  2180-04-27        1     Weight (Lbs)           94
2    10000032  2180-05-07        1      BMI (kg/m2)         18.0
3    10000032  2180-05-07        1  Height (Inches)           60
4    10000032  2180-05-07        1     Weight (Lbs)        92.15
        subject_id   chartdate  seq_num     result_name result_value
100000    10134792  2141-02-15        1  Blood Pressure       124/77
100001    10134792  2141-02-15        1     BMI (kg/m2)         34.4
100002    10134792  2141-02-15        1    Weight (Lbs)          173
100003    10134792  2141-06-25        1  Blood Pressure       122/72
100004    10134792  2141-09-17        1  Blood Pressure       146/82
        subject_id   chartdate  seq_num      result_name result_value
200000    10266717  2130-06-19        1  Height (Inches)         64.5
200001    10266717  2130-06-19        1     Weight (Lbs)

In [4]:
import pandas as pd

# Kolom yang ingin diambil
columns_to_keep = ['subject_id', 'chartdate', 'result_name', 'result_value']

# Baca file CSV dalam chunk
chunk_size = 100000  # Sesuaikan ukuran chunk sesuai kebutuhan
chunks = pd.read_csv(omr, chunksize=chunk_size, usecols=columns_to_keep)

# List untuk menyimpan hasil
processed_data = []

# Proses setiap chunk
for i, chunk in enumerate(chunks):
    print(f"Processing chunk {i + 1}...")
    
    # Normalisasi kolom 'result_name' (hilangkan spasi dan ubah ke huruf kecil)
    chunk['result_name'] = chunk['result_name'].str.strip().str.lower()
    
    # Konversi kolom 'result_value' ke tipe data numerik (hanya untuk 'weight', 'height', dan 'bmi')
    chunk.loc[chunk['result_name'].str.contains('weight|height|bmi', case=False), 'result_value'] = pd.to_numeric(
        chunk.loc[chunk['result_name'].str.contains('weight|height|bmi', case=False), 'result_value'], errors='coerce'
    )
    
    # Filter data:
    # Hanya ambil baris dengan result_name mengandung 'weight', 'height', 'blood pressure', atau 'bmi'
    filtered_chunk = chunk[
        chunk['result_name'].str.contains('weight|height|blood pressure|bmi', case=False)
    ]
    
    # Cetak beberapa baris dari chunk yang telah difilter untuk debugging
    print(f"Filtered chunk {i + 1} sample:")
    print(filtered_chunk.head())
    
    # Simpan hasil ke list
    processed_data.append(filtered_chunk)

# Gabungkan semua chunk yang telah diproses
final_df = pd.concat(processed_data, ignore_index=True)

# Simpan hasil ke file CSV (opsional)
final_df.to_csv('C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/preprocess/filtered_omr.csv', index=False)

# Tampilkan hasil
print("Final DataFrame:")
print(final_df.head())

Processing chunk 1...
Filtered chunk 1 sample:
   subject_id   chartdate      result_name result_value
0    10000032  2180-04-27   blood pressure       110/65
1    10000032  2180-04-27     weight (lbs)         94.0
2    10000032  2180-05-07      bmi (kg/m2)         18.0
3    10000032  2180-05-07  height (inches)         60.0
4    10000032  2180-05-07     weight (lbs)        92.15
Processing chunk 2...
Filtered chunk 2 sample:
        subject_id   chartdate     result_name result_value
100000    10134792  2141-02-15  blood pressure       124/77
100001    10134792  2141-02-15     bmi (kg/m2)         34.4
100002    10134792  2141-02-15    weight (lbs)        173.0
100003    10134792  2141-06-25  blood pressure       122/72
100004    10134792  2141-09-17  blood pressure       146/82
Processing chunk 3...
Filtered chunk 3 sample:
        subject_id   chartdate      result_name result_value
200000    10266717  2130-06-19  height (inches)         64.5
200001    10266717  2130-06-19     weight

In [5]:
print(final_df.tail())

         subject_id   chartdate     result_name result_value
7752743    19999828  2148-02-26  blood pressure       115/79
7752744    19999828  2148-04-29  blood pressure       105/67
7752745    19999828  2148-07-22  blood pressure       104/76
7752746    19999828  2148-10-19  blood pressure       112/73
7752747    19999828  2148-11-23  blood pressure       108/80


In [5]:
import pandas as pd

# Path ke file
filtered_omr_path = 'C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/preprocess/filtered_omr.csv'

# Load data dari file CSV
filtered_omr = pd.read_csv(filtered_omr_path)

# Filter data untuk mengambil BMI, Weight, Height, dan Blood Pressure
required_data = filtered_omr[
    filtered_omr['result_name'].isin([
        'bmi (kg/m2)', 'weight (lbs)', 'height (inches)', 'blood pressure', 'eGFR'
    ])
]

# Pivot data untuk menjadikan result_name sebagai kolom
pivoted_data = required_data.pivot_table(
    index=['subject_id', 'chartdate'],  # Gunakan subject_id dan chartdate sebagai indeks
    columns='result_name',             # Jadikan result_name sebagai kolom
    values='result_value',             # Nilai yang akan diisi ke kolom baru
    aggfunc='first'                    # Ambil nilai pertama jika ada duplikat
).reset_index()

# Simpan hasil ke file CSV baru
output_path = "C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/preprocess/pivoted_omr_vitals.csv"
pivoted_data.to_csv(output_path, index=False)

# Tampilkan hasil
print("Pivoted Data:")
print(pivoted_data.head())

Pivoted Data:
result_name  subject_id   chartdate blood pressure bmi (kg/m2)  \
0              10000032  2180-04-27         110/65         NaN   
1              10000032  2180-05-07            NaN        18.0   
2              10000032  2180-05-25         106/60        18.6   
3              10000032  2180-06-01         121/77        18.7   
4              10000032  2180-06-22         100/60        18.9   

result_name height (inches) weight (lbs)  
0                       NaN         94.0  
1                      60.0        92.15  
2                       NaN         95.0  
3                       NaN         95.7  
4                      60.0         97.0  


In [6]:
import pandas as pd

# Path ke file
pivoted_omr_path = 'C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/preprocess/pivoted_omr_vitals.csv'
admissions_complete_path = 'C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/preprocess/admissions_4.csv'

# Load data dari file CSV
pivoted_data = pd.read_csv(pivoted_omr_path)
admissions_complete = pd.read_csv(admissions_complete_path)

# Pastikan kolom 'chartdate' di admissions_complete memiliki format yang sama dengan di pivoted_data
admissions_complete['chartdate'] = pd.to_datetime(admissions_complete['chartdate']).dt.strftime('%Y-%m-%d')
pivoted_data['chartdate'] = pd.to_datetime(pivoted_data['chartdate']).dt.strftime('%Y-%m-%d')

# Gabungkan data berdasarkan subject_id dan chartdate
merged_data = pd.merge(
    admissions_complete,
    pivoted_data,
    on=['subject_id', 'chartdate'],
    how='left'  # Gunakan 'left' untuk mempertahankan semua baris dari admissions_complete
)

# Simpan hasil ke file CSV baru
output_path = 'C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/preprocess/merged_admissions_vitals.csv'
merged_data.to_csv(output_path, index=False)

# Tampilkan hasil
print("Merged Data:")
print(merged_data.head())

C:\Users\Alan Jose\AppData\Local\Temp\ipykernel_1224\1059547720.py:9: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  admissions_complete = pd.read_csv(admissions_complete_path)


Merged Data:
   subject_id   hadm_id   chartdate icd_procedure  icd_pver        admittime  \
0    10000032  22595853  2180-05-07          5491       9.0  11/21/2160 1:56   
1    10000032  22841357  2180-06-27          5491       9.0  12/28/2160 5:11   
2    10000032  25742920  2180-08-06          5491       9.0  11/15/2181 2:05   
3    10000032  29079034         NaN           NaN       NaN  9/18/2183 18:10   
4    10000068  25022803  2160-03-03          8938       9.0   8/15/2193 1:01   

          dischtime blood pressure  bmi (kg/m2)  height (inches)  weight (lbs)  
0  11/25/2160 14:52            NaN         18.0             60.0         92.15  
1  12/28/2160 16:07            NaN         19.2              NaN         98.32  
2  11/15/2181 14:52            NaN         18.2              NaN         93.03  
3   9/21/2183 16:30            NaN          NaN              NaN           NaN  
4   8/17/2193 15:07            NaN          NaN              NaN           NaN  


In [11]:
import pandas as pd

# Path ke file
pivoted_omr_path = 'C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/preprocess/pivoted_omr_vitals.csv'
admissions_complete_path = 'C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/preprocess/admissions.csv'

# Load data
pivoted_data = pd.read_csv(pivoted_omr_path)
admissions_complete = pd.read_csv(admissions_complete_path)

# Konversi ke datetime
admissions_complete['admittime'] = pd.to_datetime(admissions_complete['admittime'])
admissions_complete['dischtime'] = pd.to_datetime(admissions_complete['dischtime'])
pivoted_data['chartdate'] = pd.to_datetime(pivoted_data['chartdate'])



In [12]:
# Fungsi untuk memproses setiap subject_id
def process_subject(subject_df):
    # Ambil data vital untuk subject_id ini
    subject_vitals = pivoted_data[pivoted_data['subject_id'] == subject_df.name]
    
    # Filter vital yang berada dalam rentang waktu admittance
    mask = (subject_vitals['chartdate'] >= subject_df['admittime']) & \
           (subject_vitals['chartdate'] <= subject_df['dischtime'])
    valid_vitals = subject_vitals[mask]
    
    # Jika ada data valid, ambil yang paling terakhir
    if not valid_vitals.empty:
        latest_vital = valid_vitals.sort_values('chartdate', ascending=False).iloc[0]
        return latest_vital
    
    return pd.Series()  # Return empty Series jika tidak ada data

In [13]:
# Gabungkan data
merged_data = admissions_complete.copy()
result_columns = pivoted_data.columns.difference(admissions_complete.columns)

# Tambahkan kolom kosong untuk hasil vital
for col in result_columns:
    merged_data[col] = None

# Proses setiap admission
for idx, admission in admissions_complete.iterrows():
    vitals = process_subject(admission)
    if not vitals.empty:
        for col in result_columns:
            merged_data.at[idx, col] = vitals[col]

# Simpan hasil
output_path = 'C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/preprocess/merged_admissions_vitals.csv'
merged_data.to_csv(output_path, index=False)

print("Merged Data:")
print(merged_data.head())

Merged Data:
   subject_id   hadm_id           admittime           dischtime deathtime  \
0    10000032  22595853 2180-05-06 22:23:00 2180-05-07 17:15:00       NaN   
1    10000032  22841357 2180-06-26 18:27:00 2180-06-27 18:49:00       NaN   
2    10000032  25742920 2180-08-05 23:44:00 2180-08-07 17:50:00       NaN   
3    10000032  29079034 2180-07-23 12:35:00 2180-07-25 17:55:00       NaN   
4    10000068  25022803 2160-03-03 23:16:00 2160-03-04 06:26:00       NaN   

  chartdate  admission_type admit_provider_id      admission_location  \
0       NaN          URGENT            P49AFC  TRANSFER FROM HOSPITAL   
1       NaN        EW EMER.            P784FA          EMERGENCY ROOM   
2       NaN        EW EMER.            P19UTS          EMERGENCY ROOM   
3       NaN        EW EMER.            P06OTX          EMERGENCY ROOM   
4  3/4/2160  EU OBSERVATION            P39NWO          EMERGENCY ROOM   

  discharge_location  ... language marital_status   race        edregtime  \
0       

In [9]:
import pandas as pd

# Path ke file
pivoted_omr_path = 'C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/preprocess/processed_omr_vitals.csv'
admissions_complete_path = 'C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/preprocess/admissions_4.csv'

# Load data dari file CSV
pivoted_data = pd.read_csv(pivoted_omr_path)
admissions_complete = pd.read_csv(admissions_complete_path)

# Pastikan kolom 'chartdate' di admissions_complete memiliki format yang sama dengan di pivoted_data
admissions_complete['chartdate'] = pd.to_datetime(admissions_complete['chartdate']).dt.strftime('%Y-%m-%d')
pivoted_data['chartdate'] = pd.to_datetime(pivoted_data['chartdate']).dt.strftime('%Y-%m-%d')

# Gabungkan data berdasarkan subject_id dan chartdate
merged_data = pd.merge(
    admissions_complete,
    pivoted_data,
    on=['subject_id', 'chartdate'],
    how='left'  # Gunakan 'left' untuk mempertahankan semua baris dari admissions_complete
)

# Simpan hasil ke file CSV baru
output_path = 'C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/preprocess/merged_admissions_vitals_1.csv'
merged_data.to_csv(output_path, index=False)

# Tampilkan hasil
print("Merged Data:")
print(merged_data.head())

C:\Users\Alan Jose\AppData\Local\Temp\ipykernel_35972\4274468597.py:9: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  admissions_complete = pd.read_csv(admissions_complete_path)


Merged Data:
   subject_id   hadm_id   chartdate icd_procedure  icd_pver        admittime  \
0    10000032  22595853  2180-05-07          5491       9.0  11/21/2160 1:56   
1    10000032  22841357  2180-06-27          5491       9.0  12/28/2160 5:11   
2    10000032  25742920  2180-08-06          5491       9.0  11/15/2181 2:05   
3    10000032  29079034         NaN           NaN       NaN  9/18/2183 18:10   
4    10000068  25022803  2160-03-03          8938       9.0   8/15/2193 1:01   

          dischtime  diastolic_bp_blood_pressure result_value_blood_pressure  \
0  11/25/2160 14:52                          NaN                         NaN   
1  12/28/2160 16:07                          NaN                         NaN   
2  11/15/2181 14:52                          NaN                         NaN   
3   9/21/2183 16:30                          NaN                         NaN   
4   8/17/2193 15:07                          NaN                         NaN   

   result_value_bmi  resu

In [1]:
import pandas as pd

# Load data
pivoted_omr_path = 'C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/preprocess/processed_omr_vitals.csv'
admissions_complete_path = 'C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/preprocess/admissions_4.csv'

In [13]:
# Pastikan kolom 'chartdate' di admissions_complete memiliki format yang sama dengan di pivoted_data
admissions_complete['chartdate'] = pd.to_datetime(admissions_complete['chartdate']).dt.strftime('%Y-%m-%d')
pivoted_data['chartdate'] = pd.to_datetime(pivoted_data['chartdate']).dt.strftime('%Y-%m-%d')

In [5]:
# Merge data dengan 3 key (subject_id, hadm_id, chartdate)
merged_data = pd.merge(
    admissions_complete_path,
    pivoted_omr_path,
    on=['subject_id', 'hadm_id', 'chartdate'],
    how='left',
    suffixes=('', '_drop')
)

# Hapus kolom duplikat jika ada
merged_data = merged_data.loc[:, ~merged_data.columns.str.endswith('_drop')]

# Simpan hasil
output_path = 'C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/preprocess/merged_admissions_vitals_2.csv'
merged_data.to_csv(output_path, index=False)

# Verifikasi kolom tekanan darah
print("\nKolom tekanan darah yang tersedia:")
print([col for col in merged_data.columns if 'bp' in col.lower()])

TypeError: Can only merge Series or DataFrame objects, a <class 'str'> was passed

In [8]:
import pandas as pd

# --- Path File ---
pivoted_omr_path = 'C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/preprocess/processed_omr_vitals.csv'
admissions_complete_path = 'C:/Users/Alan Jose/Downloads/mimic-iv-3.1/mimic-iv-3.1/hosp/preprocess/admissions_4.csv'

# --- Load Data ---
pivoted_data = pd.read_csv(pivoted_omr_path)
admissions_complete = pd.read_csv(admissions_complete_path)

# --- Pastikan Format Tanggal Sesuai ---
pivoted_data['chartdate'] = pd.to_datetime(pivoted_data['chartdate'])
admissions_complete['chartdate'] = pd.to_datetime(admissions_complete['chartdate'])

# --- Merge Berdasarkan subject_id ---
merged = admissions_complete.merge(
    pivoted_data,
    on='subject_id',
    how='left',
    suffixes=('_admit', '_omr')
)

# --- Hitung Selisih Tanggal ---
merged['date_diff'] = (merged['chartdate_admit'] - merged['chartdate_omr']).abs()

# --- Batasi pada Jarak Maksimal ±3 Hari ---
merged = merged[merged['date_diff'] <= pd.Timedelta(days=3)]

# --- Ambil Data dengan chartdate OMR Terdekat ---
merged = merged.sort_values(by=['subject_id', 'chartdate'])
merged_final = merged.drop_duplicates(subset=['subject_id', 'chartdate'], keep='first')

# --- Pilih Kolom Akhir ---
cols_to_keep = [
    'subject_id', 'hadm_id', 'chartdate',
    'result_value_blood_pressure','result_value_bmi','result_value_egfr',
    'result_value_height','result_value_weight'

]
merged_final = merged_final[cols_to_keep]

# --- Simpan ke File ---
merged_final.to_csv(output_path, index=False)

# --- Tampilkan Hasil ---
print("✅ Merged data successfully saved!")
print(merged_final.head())


C:\Users\Alan Jose\AppData\Local\Temp\ipykernel_35972\627918571.py:9: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  admissions_complete = pd.read_csv(admissions_complete_path)


KeyError: 'chartdate'